In [0]:
from __future__ import print_function
import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
      def __init__(self):
          super(Net, self).__init__()
          self.conv1 = nn.Sequential(
              nn.Conv2d(in_channels=1,out_channels= 16,kernel_size=(3,3),padding=0,bias=False),
              nn.ReLU(),
              nn.BatchNorm2d(16),
              #nn.Conv2d(32, 32, 1, padding=0),
              #nn.ReLU(),
              #nn.BatchNorm2d(32),
              nn.Dropout(0.2)
          )
          self.conv2 = nn.Sequential(
              nn.Conv2d(in_channels=16,out_channels=32,kernel_size=(3,3),padding=0,bias=False),
              nn.ReLU(),
              nn.BatchNorm2d(32),
              #nn.Conv2d(64, 10, 1, stride=1, padding=0),
              #nn.BatchNorm2d(10),
              #nn.MaxPool2d(2, 2),
              nn.Dropout(0.2)
          )
          
          self.conv3 = nn.Sequential(
              nn.Conv2d(in_channels=32,out_channels=10,kernel_size=(1,1), padding=0,bias=False))
              #nn.ReLU(),
              #nn.BatchNorm2d(128),
          self.pool1= nn.MaxPool2d(2, 2)
              
          
          
          self.conv4=nn.Sequential(
            nn.Conv2d(in_channels=10,out_channels=16,kernel_size=(3,3),padding=0,bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.2)
        )
          self.conv5=nn.Sequential(
            nn.Conv2d(in_channels=16,out_channels=16,kernel_size=(3,3),padding=0,bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.2)
        )

          self.conv6=nn.Sequential(
            nn.Conv2d(in_channels=16,out_channels=16,kernel_size=(3,3),padding=0,bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.2)
        )
          self.conv7=nn.Sequential(
            nn.Conv2d(in_channels=16,out_channels=16,kernel_size=(3,3),padding=1,bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.2)       )
          
          self.gap=nn.Sequential(
              nn.AvgPool2d(6)
              )
          
          self.conv8=nn.Sequential(
              nn.Conv2d(in_channels=16,out_channels=10,kernel_size=(1,1),padding=0,bias=False)
          )
   #      self.dropout=nn.Dropout(0.2)
  
      def forward(self, x):
          x = self.conv1(x)
          x = self.conv2(x)
          x = self.conv3(x)
          x = self.pool1(x)
          x = self.conv4(x)
          x = self.conv5(x)
          x = self.conv6(x)
          x = self.conv7(x)
          x = self.gap(x)
          x = self.conv8(x)
          x = x.view(-1,10)
          x = F.log_softmax(x, dim=1)
          return x      

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,608
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
           Dropout-8           [-1, 32, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             320
        MaxPool2d-10           [-1, 10, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,440
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
          Dropout-14           [-1, 16,

In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1354, Accuracy: 9596/10000 (95.96%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0535, Accuracy: 9837/10000 (98.37%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0209, Accuracy: 9935/10000 (99.35%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0203, Accuracy: 9938/10000 (99.38%)



loss=0.06723446398973465 batch_id=468: 100%|██████████| 469/469 [00:10<00:00, 43.47it/s]



Test set: Average loss: 0.0200, Accuracy: 9933/10000 (99.33%)

